# Mohammed


In [1]:
import numpy as np
import os
import io
import tensorflow as tf
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
import random
from ipywidgets import interact, interactive, fixed, interact_manual, VBox, HBox
from PIL import Image
from keras.models import load_model

In [2]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


def reconstruct_image(image,  vae_model):
    image = image / 255.0
    image = np.expand_dims(image, axis=2)
    reconstructed_image = vae_model.predict(image[None, ...])[0]
    mse = mean_squared_error(image, reconstructed_image )
    print("MSE = ", mse)
    return reconstructed_image.squeeze(axis=2)


def load_my_model(path):
    model = load_model(path)
    return model


vae = load_my_model("path to vae_model.h5")
vae_task2 = load_my_model("path to vae_task2_model.h5")


## Please rerun cell 3 again if you encounter error while interacting, it will perfrom correctly then

In [5]:
state = {}
state['vae1'] = vae

state['latent_values'] = [0.0] * 6
state['sliders'] = [
    widgets.FloatSlider(min=-3, max=3, step=0.1, description=f'Dim {i+1}')
    for i in range(6)
]
state['image_widget'] = widgets.Image(format='png', width=300, height=400)


def update_image(change):
    latent_vector = np.array([[slider.value for slider in state['sliders']]])
    reconstructed_img = state['vae1'].get_layer('decoder').predict(latent_vector)
    reconstructed_img = np.squeeze(reconstructed_img, axis=0)
    reconstructed_img = (reconstructed_img * 255).astype(np.uint8)

    if reconstructed_img.ndim == 3:
        reconstructed_img = reconstructed_img[:, :, 0]

    img = Image.fromarray(reconstructed_img, 'L')
    with io.BytesIO() as output:
        img.save(output, format="PNG")
        contents = output.getvalue()
    state['image_widget'].value = contents


for slider in state['sliders']:
    slider.observe(update_image, names='value')


sliders_vbox = widgets.VBox(state['sliders'])
ui = widgets.HBox([sliders_vbox, state['image_widget']])
display(ui)

update_image(None)


1/1 [==============================] - 0s 17ms/step


In [4]:
def to_array(image):

    image = Image.open(io.BytesIO(image))
    arr = np.array(image)
    return arr

def array_to_bytes(arr):
 
    
    if arr.max() <= 1:
        image = Image.fromarray((arr * 255).astype(np.uint8))
    else:
        image = Image.fromarray((arr).astype(np.uint8))
 
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='jpeg')

    img_byte_arr = img_byte_arr.getvalue()

    return img_byte_arr


state = {}

state['image'] = np.ones((48,48))
state['masked_image'] = np.copy(state['image'])
state['upload_file'] = widgets.FileUpload(
        accept='jpg',
        multiple=False
    )

state['vae1'] = vae
state['vae2'] = vae_task2



image_np = np.array(state['image'])
masked_image = np.copy(image_np)




size_factor_slider = widgets.FloatSlider(value=0., min=0., max=.5, step=0.05, description='Size Factor:')
start_y_slider = widgets.FloatSlider(value=0., min=0., max=1., step=0.05, description='Start X:')
start_x_slider = widgets.FloatSlider(value=0., min=0., max=1., step=0.05, orientation='vertical' , description=' Start Y:')



output_widget_1 = widgets.Output()
output_widget_2 = widgets.Output()
output_widget_3 = widgets.Output()



def update_image(change):

        start_x = start_x_slider.value
        start_y = start_y_slider.value
        size_factor = size_factor_slider.value


        shape = state['image'].shape



        masked_image = np.copy(state['image'])
        mask_height = int(size_factor * shape[0])
        mask_width = int(size_factor * shape[1])


        start_x = int(start_x  *  (shape[0] - mask_height))
        start_y = int(start_y  *  (shape[1] - mask_width))

        end_x = int( start_x + mask_height )
        end_y = int( start_y + mask_width )
        masked_image[start_x:end_x , start_y:end_y] = 0

        with output_widget_1:
            clear_output(wait=True)


            display(widgets.Image(
                      value=array_to_bytes(masked_image),
                      format='jpeg',
                      width=300,
                      height=400,
                  ))

        with output_widget_2:
            clear_output(wait=True)

            display(widgets.Image(
                      value=array_to_bytes(reconstruct_image(masked_image, state['vae1'])),
                      format='jpeg',
                      width=300,
                      height=400,
                  ))

        with output_widget_3:
            clear_output(wait=True)

            display(widgets.Image(
                      value=array_to_bytes(reconstruct_image(masked_image, state['vae2'])),
                      format='jpg',
                      width=300,
                      height=400,
                  ))




start_x_slider.observe(update_image, names='value')
start_y_slider.observe(update_image, names='value')
size_factor_slider.observe(update_image, names='value')


def show_file_upload(event):
      display(state['upload_file'])


def on_upload_change(change):

    uploader = state['upload_file']
    uploaded_file = uploader.value[0]['content'].tobytes()
    state['image'] = to_array(uploaded_file)

    update_image("")


state['upload_file'].observe(on_upload_change, names='value')


upload_button = widgets.Button(description="upload a file")
upload_button.on_click(show_file_upload)
image_load = VBox([output_widget_1, upload_button])





internal = HBox([start_x_slider, image_load ])
col1 = VBox([size_factor_slider, start_y_slider,   internal])




col2 = VBox([output_widget_2])
col3 = VBox([output_widget_3])



main_box = HBox([col1, col2, col3])
update_image("")
display(main_box)
